In [2]:
from pyspark.sql import SparkSession

# Create SparkSession connecting to the Spark cluster (NOT Spark Connect)
# This approach allows for true distributed data loading
spark = (
    SparkSession.builder.appName("PurchaseIntentionAnalysis")
    .remote("sc://192.168.1.7:15002")  # Direct connection to Spark master (not Connect)
    .config("spark.sql.ansi.enabled", "false")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "1g")
    .getOrCreate()
)

# Enable eager evaluation for nice HTML tables in Jupyter
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

c:\Users\plancha\purchase-intent\.venv\Lib\site-packages\pyspark\sql\connect\conf.py:64: UserWarning: Failed to set spark.executor.memory to Some(2g) due to [CANNOT_MODIFY_CONFIG] Cannot modify the value of the Spark config: "spark.executor.memory".
See also 'https://spark.apache.org/docs/latest/sql-migration-guide.html#ddl-statements'. SQLSTATE: 46110
  warnings.warn(warn)
c:\Users\plancha\purchase-intent\.venv\Lib\site-packages\pyspark\sql\connect\conf.py:64: UserWarning: Failed to set spark.driver.memory to Some(1g) due to [CANNOT_MODIFY_CONFIG] Cannot modify the value of the Spark config: "spark.driver.memory".
See also 'https://spark.apache.org/docs/latest/sql-migration-guide.html#ddl-statements'. SQLSTATE: 46110
  warnings.warn(warn)


In [12]:
# Read parquet files from all three locations (master + 2 workers)
# The master can access all three directories, but workers can only access their own
sessions_data = spark.read.csv(["/opt/spark/data/worker1/*.csv", "/opt/spark/data/worker2/*.csv"], header=True, inferSchema=True)
sessions_data.createOrReplaceTempView("sessions_data")
sessions_data

Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0,0.0,22,370.3333333,0.018181818,0.054545455,0.0,0.6,May,2,2,4,13,Returning_Visitor,false,false
12,381.5,1,22.2,154,7835.874629,0.012549679,0.022587818,0.0,0.0,Aug,3,2,1,2,Returning_Visitor,true,false
5,44.75,3,51.5,107,3074.852778,0.015454545,0.026239965,0.0,0.0,Nov,2,2,3,1,Returning_Visitor,true,false
1,12.0,0,0.0,11,213.0,0.05,0.066666667,0.0,0.0,Nov,2,2,4,13,Returning_Visitor,true,false
0,0.0,0,0.0,1,0.0,0.2,0.2,0.0,0.0,May,1,1,3,3,New_Visitor,false,false
1,18.0,1,16.0,33,504.0,0.006060606,0.033333333,0.0,0.0,May,2,4,1,4,Returning_Visitor,false,false
1,51.4,0,0.0,7,562.3,0.0,0.0,36.65735004,0.0,Jul,1,1,6,2,New_Visitor,true,true
2,46.4,0,0.0,8,349.0,0.02,0.08,0.0,0.0,Aug,4,1,1,1,Returning_Visitor,false,false
4,54.4,0,0.0,68,2889.946154,0.002898551,0.008789401,0.0,0.0,June,4,1,4,1,Returning_Visitor,false,false
3,335.5,1,15.0,4,96.0,0.0,0.025,0.0,0.0,Nov,1,1,8,15,Returning_Visitor,false,false


In [15]:
spark.sql("select count(*) from sessions_data")

count(1)
12330
